In [6]:
import pandas as pd
import requests
import string

In [7]:
# Import data
data_init = pd.read_csv('../data/nlp-data.csv')

In [32]:
# Dataframe containg the data that our team created
data_init.head()

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
0,Assign completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
1,Move X to completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
2,X completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
3,Completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
4,Started working on X,Task,X,In Progress,"[1,0,0]","[0,1,0,0,0]"


In [27]:
# Script for creating new data by substituting words for their synonyms. Only the words which are not included
# in the identifier are eligible for being replaced
def aug_data(df):
    #new dataframe for the new sentences
    added_data = pd.DataFrame(columns=list(df.columns))
    index = len(df)
    for i in range(len(df.index)):
        identifier = df.iloc[i][2]
        text = df.iloc[i][0]
        sentence_lst = text.split()
        identifier_lst = identifier.split()
        for j in range(len(sentence_lst)):
            word = sentence_lst[j]
            if word not in identifier_lst:
                word = word.lower()
                api_url = 'https://api.datamuse.com/words?rel_syn={0}&max=5'.format(word)
                word_synonyms = requests.get(api_url)
                word_synonyms = word_synonyms.json()
                for k in range(len(word_synonyms)):
                    new_sentence_lst = sentence_lst[:j]+[word_synonyms[k]['word']]+sentence_lst[j+1:]
                    s = " "
                    new_sentence = s.join(new_sentence_lst)
                    added_data.loc[index] = [new_sentence] + list(df.loc[i][1:])
                    index += 1
    augmented_data = pd.concat([df,added_data])
    return augmented_data

In [ ]:
# Limiting data to just completed tasks for API exploration purposes
data_init = data_init.

In [28]:
new_data = aug_data(data_init)

In [29]:
new_data

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
0,Assign completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
1,Move X to completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
2,X completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
3,Completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
4,Started working on X,Task,X,In Progress,"[1,0,0]","[0,1,0,0,0]"
...,...,...,...,...,...,...
23,Create X,Task,X,To Do,"[1,0,0]","[1,0,0,0,0]"
24,Completed a X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
25,Completed an X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
26,Create a X,Task,X,To Do,"[1,0,0]","[1,0,0,0,0]"


In [25]:
pd.set_option("display.max_rows", 10)